In [5]:
import pandas as pd
import numpy as np

In [58]:
df = pd.read_csv('/home/jupyter-vladperesad/files/THE QUERY-.csv')
df

,student_id,student_name,group_name,date,homework,behaviour,comprehension,vocabulary,speaking,reading,writing
0,1,Ethan,wed1905,2024-02-28,0.0,5,3,3,2,3.0,NaN
1,2,Dora,wed1905,2024-02-28,0.0,5,5,5,4,5.0,NaN
2,3,Eleven,wed1905,2024-02-28,5.0,5,5,4,2,4.0,NaN
3,4,Mason,thu1735,2024-02-29,NaN,5,5,5,4,5.0,5.0
4,5,Elsa,thu1735,2024-02-29,NaN,4,5,2,2,3.0,2.0
5,6,David,thu1735,2024-02-29,NaN,5,5,2,3,5.0,4.0
6,7,Wilson,thu1905,2024-02-22,5.0,4,4,5,4,4.0,NaN
7,7,Wilson,thu1905,2024-02-29,5.0,2,5,5,5,5.0,5.0
8,8,Evan,thu1905,2024-02-22,0.0,3,2,4,4,3.0,NaN
9,8,Evan,thu1905,2024-02-29,0.0,2,4,3,2,5.0,5.0


In [53]:
df_2 = df.drop(columns=['student_name',
                        'group_name',
                        'date'])

In [59]:
df_2 = df \
    .drop(columns=['student_name',
                    'group_name',
                    'date']) \
    .groupby('student_id') \
    .pct_change(fill_method=None)*100

In [60]:
df_2.dropna(axis=0, how='all', inplace=True)

In [61]:
df.join(df_2, how='inner', rsuffix= '_pct')

,student_id,student_name,group_name,date,homework,behaviour,comprehension,vocabulary,speaking,reading,writing,homework_pct,behaviour_pct,comprehension_pct,vocabulary_pct,speaking_pct,reading_pct,writing_pct
7,7,Wilson,thu1905,2024-02-29,5.0,2,5,5,5,5.0,5.0,0.0,-50.000000,25.000000,0.000000,25.000000,25.000000,NaN
9,8,Evan,thu1905,2024-02-29,0.0,2,4,3,2,5.0,5.0,NaN,-33.333333,100.000000,-25.000000,-50.000000,66.666667,NaN
11,9,Daniel,thu1905,2024-02-29,0.0,4,2,4,2,3.0,3.0,NaN,33.333333,-60.000000,0.000000,-50.000000,200.000000,NaN
15,13,Charles,fri1905,2024-03-01,0.0,1,5,4,5,NaN,1.0,NaN,-50.000000,0.000000,100.000000,0.000000,NaN,-80.0
21,19,Robert,sat1040,2024-03-02,3.0,5,5,5,5,NaN,3.0,-40.0,0.000000,0.000000,0.000000,0.000000,NaN,-25.0
23,20,Elsa,sat1040,2024-03-02,0.0,5,5,5,5,NaN,2.0,NaN,0.000000,0.000000,25.000000,0.000000,NaN,0.0
26,22,Claire,sat1040,2024-03-02,3.0,3,5,5,4,NaN,2.0,inf,-40.000000,66.666667,66.666667,33.333333,NaN,inf
29,24,Alex,sat1400,2024-03-02,NaN,4,5,5,5,4.0,NaN,NaN,-20.000000,66.666667,0.000000,0.000000,0.000000,NaN
33,27,Jill,sat1400,2024-03-02,NaN,4,4,4,5,4.0,NaN,NaN,-20.000000,33.333333,-20.000000,25.000000,0.000000,NaN
35,28,Ryan,sat1400,2024-03-02,NaN,0,5,0,2,5.0,NaN,NaN,-100.000000,66.666667,-100.000000,-33.333333,25.000000,NaN
